In [8]:
import pandas as pd

In [33]:
#loading data

data_BIS = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_BIS.csv", encoding = "UTF-8")
data_Yeoshin = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_Yeoshin.csv", encoding = "UTF-8")
data_ROA = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_ROA.csv", encoding = "UTF-8")
data_self = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_self.csv", encoding = "UTF-8")

data_BIS = data_BIS.rename(columns = {'a' : 'BIS'})
data_Yeoshin = data_Yeoshin.rename(columns = {'a' : 'Yeoshin'})
data_ROA = data_ROA.rename(columns = {'a' : 'ROA'})
data_self = data_self.rename(columns = {'a' : 'self'})

In [34]:
#merging two dataframes

record_id_BIS = []
record_id_Yeoshin = []
record_id_ROA = []
record_id_self = []

for i in range(data_BIS.shape[0]):
    record_id_BIS.append(str(data_BIS.iloc[i]['finance_cd']) + ' ' + str(data_BIS.iloc[i]['base_month']))

for i in range(data_Yeoshin.shape[0]):
    record_id_Yeoshin.append(str(data_Yeoshin.iloc[i]['finance_cd']) + ' ' + str(data_Yeoshin.iloc[i]['base_month']))

for i in range(data_ROA.shape[0]):
    record_id_ROA.append(str(data_ROA.iloc[i]['finance_cd']) + ' ' + str(data_ROA.iloc[i]['base_month']))    
    
for i in range(data_self.shape[0]):
    record_id_self.append(str(data_self.iloc[i]['finance_cd']) + ' ' + str(data_self.iloc[i]['base_month']))    
    
    
data_BIS['record_id'] = record_id_BIS
data_Yeoshin['record_id'] = record_id_Yeoshin
data_ROA['record_id'] = record_id_ROA
data_self['record_id'] = record_id_self

Merged_BIS_Yeoshin = pd.merge(data_BIS, data_Yeoshin, how="inner", left_on = "record_id", right_on = "record_id")
Merge_3 = pd.merge(Merged_BIS_Yeoshin, data_ROA, how="inner", left_on = "record_id", right_on = "record_id")
Merge_4 = pd.merge(Merge_3, data_self, how="inner", left_on = "record_id", right_on = "record_id")

Merge_4 = Merge_4[['base_month_x', 'finance_cd_x', 'finance_nm_x', 'BIS', 'Yeoshin', 'ROA', 'self']]
Merge_4 = Merge_4.ix[:,[0,2,4,6,7,8,9]]


# delete March, September
check_month = []

for i in range(Merge_4.shape[0]):
    if Merge_4.iloc[i]['base_month_x'].split(" ")[0] == "3" or Merge_4.iloc[i]['base_month_x'].split(" ")[0] == "9":
        check_month.append(i)

deleted_Merge_4 = Merge_4.drop(check_month, 0)

# sort by company, date
date=[]

for i in range(deleted_Merge_4.shape[0]):
    if deleted_Merge_4.iloc[i]['base_month_x'].split(" ")[0] == "12":
        temp = deleted_Merge_4.iloc[i]['base_month_x'].split(" ")[1] + "." + deleted_Merge_4.iloc[i]['base_month_x'].split(" ")[0]
    elif deleted_Merge_4.iloc[i]['base_month_x'].split(" ")[0] == "6":
        temp = deleted_Merge_4.iloc[i]['base_month_x'].split(" ")[1] + ".0" + deleted_Merge_4.iloc[i]['base_month_x'].split(" ")[0]
    date.append(float(temp))
    
deleted_Merge_4['date'] = date
sorted_Merge_4 = deleted_Merge_4.sort_values(['finance_cd_x', 'date'], ascending =[True, True])


# make predict variable "weak"
weak = []
for i in range(sorted_Merge_4.shape[0]-1):
    if sorted_Merge_4.iloc[i+1]['finance_cd_x'] == sorted_Merge_4.iloc[i]['finance_cd_x']:    
        if float(sorted_Merge_4.iloc[i+1]['self']) < 4:
            weak.append(1)
        else:
            weak.append(0)
    else:
        weak.append(2)
weak.append(2)
sorted_Merge_4['weak'] = weak
final_data = sorted_Merge_4[sorted_Merge_4.weak != 2]

# change attribute's type to float64
final_data['BIS'] = final_data['BIS'].apply(pd.to_numeric)


#final_data.to_csv("sorted_data.csv")

In [161]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [166]:
#get X, Y for training

X = final_data[['ROA', 'BIS', 'Yeoshin']]
Y = final_data['weak']

# seperate training and test set
X_train, X_test, y_train, y_test= cross_validation.train_test_split(X, Y, test_size = .3, random_state = 1234)

# normalizing
'''
sc = StandardScaler()
sc.fit(X_train)
X_train= sc.transform(X_train)
X_test= sc.transform(X_test)
'''



'\nsc = StandardScaler()\nsc.fit(X_train)\nX_train= sc.transform(X_train)\nX_test= sc.transform(X_test)\n'

In [167]:

# start logistic regression training
lr = LogisticRegression(random_state = 1234)
lr.fit(X_train, y_train)

# show scores
print("train_score : ", lr.score(X_train, y_train))
print("test_score : ", lr.score(X_test, y_test))

y_test_pred = lr.predict(X_test)
y_train_pred = lr.predict(X_train)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
print(test_accuracy, train_accuracy)

print("Classification report for test set...")
print(metrics.classification_report(y_test, y_test_pred))

print("Classification report for training set...")
print(metrics.classification_report(y_train, y_train_pred))

# 행:실제 분류   열: 분류기 분류
print("Confusion matrix for test set...")
print(metrics.confusion_matrix(y_test, y_test_pred))

print("Confusion matrix for training set...")
print(metrics.confusion_matrix(y_train, y_train_pred))

train_score :  0.853503184713
test_score :  0.899109792285
0.899109792285 0.853503184713
Classification report for test set...
             precision    recall  f1-score   support

          0       0.90      0.99      0.94       288
          1       0.86      0.37      0.51        49

avg / total       0.90      0.90      0.88       337

Classification report for training set...
             precision    recall  f1-score   support

          0       0.87      0.97      0.91       636
          1       0.74      0.36      0.48       149

avg / total       0.84      0.85      0.83       785

Confusion matrix for test set...
[[285   3]
 [ 31  18]]
Confusion matrix for training set...
[[617  19]
 [ 96  53]]


In [168]:
# random forest training

rf = RandomForestClassifier(criterion="entropy", n_estimators=50, max_depth = 4,random_state = 0  )
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [169]:
# show scores
print("train_score : ", rf.score(X_train, y_train))
print("test_score : ", rf.score(X_test, y_test))

y_test_pred = rf.predict(X_test)
y_train_pred = rf.predict(X_train)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
print(test_accuracy, train_accuracy)

print("Classification report for test set...")
print(metrics.classification_report(y_test, y_test_pred))

print("Classification report for training set...")
print(metrics.classification_report(y_train, y_train_pred))

# 행:실제 분류   열: 분류기 분류
print("Confusion matrix for test set...")
print(metrics.confusion_matrix(y_test, y_test_pred))

print("Confusion matrix for training set...")
print(metrics.confusion_matrix(y_train, y_train_pred))

train_score :  0.922292993631
test_score :  0.910979228487
0.910979228487 0.922292993631
Classification report for test set...
             precision    recall  f1-score   support

          0       0.95      0.94      0.95       288
          1       0.69      0.71      0.70        49

avg / total       0.91      0.91      0.91       337

Classification report for training set...
             precision    recall  f1-score   support

          0       0.95      0.95      0.95       636
          1       0.80      0.79      0.79       149

avg / total       0.92      0.92      0.92       785

Confusion matrix for test set...
[[272  16]
 [ 14  35]]
Confusion matrix for training set...
[[606  30]
 [ 31 118]]
